# NCEDC phase & polarity picks combine with PhaseNet picks and DiTingMotion polarities
NCEDC has a good number of picks and polarities,
but usually no picks for stations of other networks.
So following steps will be taken to get missing picks and polarities using PhaseNet and DiTingMotion:

1. Use PhaseNet on all events to get picks for all stations.
2. Keep only the best picks from PhaseNet using SNR and Standard Deviation.
3. Use DiTingMotion to get polarities.
4. Combine NCEDC picks and polarities with PhaseNet picks and DiTingMotion polarities.
5. Write the combined picks and polarities to PyRocko format.

In [1]:
#!/Users/mdaislam/anaconda3/envs/phasenet
import os
import glob
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import obspy
from obspy.clients.fdsn import Client
from obspy import UTCDateTime, read_inventory, Inventory, read, Stream


In [2]:
# Function to list files in a directory
def my_list_files(directory, file_extension=None):
    import glob
    file_path_list = glob.glob(f'{directory}/*{file_extension}')
    file_list = sorted([os.path.basename(f) for f in file_path_list])
    return file_list

In [3]:
research = os.path.abspath('../../../..')
data_dir = f'{research}/Data/NCEDC_events_data'
phasenet_git_dir = f'{research}/Software_docs/PhaseNet'
nc_picks_dir = f'{research}/Github/FM2STRESS/FM2STRESS_project/data/NCEDC_picks'
eq_cat_file = f'{nc_picks_dir}/NCEDC_eq_cat_above_slab.csv'
sta_inv_file = ''
out_dir = os.path.abspath(os.path.join(
        os.getcwd(), '../results/phasenet_diting_others/phasenet_picks_NCEDC_cat'))

# Go by year and run PhaseNet
years_dirs = my_list_files(f'{data_dir}/mseed', file_extension='')
print(years_dirs)
print(os.listdir(nc_picks_dir))

['2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023']
['2008_NCEDC_picks_above_slab.csv', '2016.txt', '2012_NCEDC_picks_above_slab.csv', '2017_NCEDC_picks_above_slab_pyrocko.txt', '2017.txt', '2015.txt', '2021_NCEDC_picks_above_slab_pyrocko.txt', 'NCEDC_picks_50km_pyrocko.txt', '2014.txt', '2010.txt', '.DS_Store', 'NCEDC_picks_above_slab_pyrocko.txt', '2011.txt', '2020_NCEDC_picks_above_slab.csv', '2013.txt', '2019_NCEDC_picks_above_slab.csv', '2011_NCEDC_picks_above_slab_pyrocko.txt', '2015_NCEDC_picks_above_slab.csv', '2008_NCEDC_picks_above_slab_pyrocko.txt', '2012.txt', '2016_NCEDC_picks_above_slab_pyrocko.txt', '2018_NCEDC_picks_above_slab.csv', 'NCEDC_picks_50km.csv', '2014_NCEDC_picks_above_slab.csv', '2021_NCEDC_picks_above_slab.csv', '2020_NCEDC_picks_above_slab_pyrocko.txt', '2009_NCEDC_picks_above_slab.csv', 'NCEDC_eq_cat_above_slab.csv', '2013_NCEDC_picks_above_slab.csv', '2010_NCEDC_picks_above_slab

# Pick P-arrivals with PhaseNet

In [4]:
from classes_functions.other_fun import make_phasenet_script

for year in years_dirs:
    print(f'Processing year: {year}')
    # Make a list of .mseed files into a csv
    mseed_files = my_list_files(f'{data_dir}/mseed/{year}', '.mseed')
    pd.DataFrame(mseed_files, columns=['fname']
        ).to_csv(f'{out_dir}/{year}_mseed_list.csv', index=False, header=True)
    
    # make bash script for running PhaseNet
    script_content = make_phasenet_script(
        phasenet_git_dir,
        waveform_dir=f'{data_dir}/mseed/{year}',
        data_list=f'{out_dir}/{year}_mseed_list.csv',
        result_dir=f'{out_dir}/{year}',
        result_fname=f'{year}_phasenet_picks',
        conda_env='minconda3',
    )

    # Write the script content to a file
    with open(f"{out_dir}/run_phasenet.sh", "w") as f:
        f.write(script_content)
    
    # Make the script executable
    print("Making the script executable...")
    os.system(f"chmod +x {out_dir}/run_phasenet.sh")
    
    # Run the script
    run = input("Do you want to run PhaseNet now? (y/n): ")
    
    if run.lower() == "y":
        print("Running PhaseNet...")
        os.system(f"{out_dir}/run_phasenet.sh")
    else:
        print("PhaseNet already run. Skipping...")
    
    
    break


Processing year: 2008
Making the script executable...
Running PhaseNet...


/Users/mdarifulislam/Library/CloudStorage/OneDrive-IndianaUniversity/Research/Github/FM2STRESS/FM2STRESS_project/results/phasenet_diting_others/phasenet_picks_NCEDC_cat/run_phasenet.sh: line 7: /Users/mdarifulislam/minconda3/etc/profile.d/conda.sh: No such file or directory

CondaError: Run 'conda init' before 'conda activate'

2024-06-03 00:34:09,846 Pred log: /Users/mdarifulislam/Library/CloudStorage/OneDrive-IndianaUniversity/Research/Github/FM2STRESS/FM2STRESS_project/results/phasenet_diting_others/phasenet_picks_NCEDC_cat/2008
2024-06-03 00:34:09,846 Dataset size: 151
2024-06-03 00:34:09,968 Model: depths 5, filters 8, filter size 7x1, pool size: 4x1, dilation rate: 1x1
2024-06-03 00:34:10.671939: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2024-06-03 00:34:10,862 restoring model model/190703-214543/model_95.ckpt
Pred: 100%|██████████| 151/151 [00:34<00:00,  4.33it/s]


Done with 5685 P-picks and 0 S-picks


# Mark good traces with SNR and Standard Deviation


In [5]:
def snr_std_for_trace(
    tr: obspy.core.trace.Trace,
    phase_time,
    width_window=0.5,
):
    ##### define signal window and noise window
    signal_window = [phase_time, phase_time+width_window]
    noise_window = [phase_time-(width_window+0.1), phase_time-0.1]

    signal = tr.slice(starttime=signal_window[0], endtime=signal_window[1])
    noise = tr.slice(starttime=noise_window[0], endtime=noise_window[1])

    sigmax = np.max(np.abs(signal.data))
    noisemax = np.max(np.abs(noise.data))

    snr = (sigmax/noisemax).round(2)

    ##### std
    # how many samples within 1 std
    tr2 = tr.slice(starttime=phase_time-1, endtime=phase_time+1)
    std = np.std(tr2.data)
    mean = np.mean(tr2.data)
    nstd = (np.sum(np.abs(tr2.data-mean) < std)/len(tr2.data)).round(2)

    return snr, nstd

### Keep picks only high SNR traces (marked by `pn_quality` = 1 [/bad=0])

In [6]:
eq_df = pd.read_csv(eq_cat_file, parse_dates=['time'])

##### Process the PhaseNet picks and Keep picks only for "good" traces
for year in years_dirs:
    # year = 2013
    # Load the PhaseNet picks
    picks_df = pd.read_csv(f'{out_dir}/{year}/{year}_phasenet_picks.csv', parse_dates=['phase_time'])
    picks_df['event_id'] = picks_df['file_name'].str.split('.').str[0]
    picks_df = picks_df.merge(
        eq_df[['id', 'time']],
        left_on='event_id',
        right_on='id',
        how='left', # 
    ).rename(columns={'time': 'event_time'}
    ).drop(columns='id')

    # Drop rows if P-arriaval is before the event time i.e. event_time > phase_time
    picks_df = picks_df[picks_df['event_time'] <= picks_df['phase_time']]
    picks_df['pn_quality'] = 0

    # Loop each event
    for event_id, event_df in picks_df.groupby('event_id'):
        st = read(f'{data_dir}/mseed/{year}/{event_id}.mseed')
        for i, row in event_df.iterrows():
            phase_time = UTCDateTime(pd.to_datetime(str(row.phase_time)))
            ist = st.select(id=f"{row.station_id}Z")
            if len(ist) == 0:
                continue
            tr = ist[0].copy()

            tr.detrend('demean')
            tr.taper(max_percentage=0.001, type='cosine')
            tr.filter('bandpass', freqmin=1.5, freqmax=10)

            snr, nstd = snr_std_for_trace(tr, phase_time)
            if snr >= 20 and nstd >= 0.7:
                picks_df.loc[i, 'pn_quality'] = 1
            # elif 5 <= snr < 20 and nstd >= 0.8:
            #     pass
            else:
                pass
    
    picks_df = picks_df[picks_df['pn_quality'] == 1]
    picks_df.to_csv(f'{out_dir}/{year}/{year}_phasenet_picks_SNR.csv', index=False)
    break

picks_df

,station_id,begin_time,phase_index,phase_time,phase_score,phase_type,file_name,phase_amplitude,phase_amp,event_id,event_time,pn_quality
1,NC.KCR..SH,2008-01-28T20:06:13.900,3571,2008-01-28 20:06:49.610,0.942,P,nc40210243.mseed,332.947723,332.947723,nc40210243,2008-01-28 20:06:43.900,1
2,NC.KCS..SH,2008-01-28T20:06:13.900,3518,2008-01-28 20:06:49.080,0.983,P,nc40210243.mseed,598.418274,598.418274,nc40210243,2008-01-28 20:06:43.900,1
3,NC.KHBB..HH,2008-01-28T20:06:13.900,3767,2008-01-28 20:06:51.570,0.982,P,nc40210243.mseed,1965.645264,1965.645264,nc40210243,2008-01-28 20:06:43.900,1
4,NC.KHMB..HH,2008-01-28T20:06:13.900,3696,2008-01-28 20:06:50.860,0.987,P,nc40210243.mseed,2022.681763,2022.681763,nc40210243,2008-01-28 20:06:43.900,1
10,NC.LGP..SH,2008-01-28T20:06:13.900,4308,2008-01-28 20:06:56.980,0.965,P,nc40210243.mseed,646.003662,646.003662,nc40210243,2008-01-28 20:06:43.900,1
...,...,...,...,...,...,...,...,...,...,...,...,...
5669,NC.KSXB..HH,2008-12-30T13:45:50.775,4825,2008-12-30 13:46:39.025,0.978,P,nc71180461.mseed,3471.261475,3471.261475,nc71180461,2008-12-30 13:46:20.770,1
5674,PB.B933..EH,2008-12-30T13:45:50.775,5468,2008-12-30 13:46:45.455,0.922,P,nc71180461.mseed,155.176193,155.176193,nc71180461,2008-12-30 13:46:20.770,1
5676,XQ.ME01.01.BH,2008-12-30T13:45:50.775,5215,2008-12-30 13:46:42.925,0.919,P,nc71180461.mseed,1159.939209,1159.939209,nc71180461,2008-12-30 13:46:20.770,1
5677,XQ.ME03.01.BH,2008-12-30T13:45:50.775,4703,2008-12-30 13:46:37.805,0.977,P,nc71180461.mseed,1977.267700,1977.267700,nc71180461,2008-12-30 13:46:20.770,1


# Polarity determination using DiTing-Motion

In [7]:
import tensorflow as tf
from keras import backend as K
from classes_functions.my_class_funcs import DitingMotionPicker

# os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'  # Suppress TensorFlow logging
diting_model_dir = os.path.abspath(os.path.join(os.getcwd(), "./DiTing_model/models/DiTingMotionJul.hdf5"))
diting_model = tf.keras.models.load_model(diting_model_dir, compile=False)

for year in years_dirs:
    print(f'Processing year: {year}')
    mseed_files = my_list_files(f'{data_dir}/mseed/{year}', '.mseed')
    picks_df = pd.read_csv(f'{out_dir}/{year}/{year}_phasenet_picks_SNR.csv', parse_dates=['phase_time'])

    dmp = DitingMotionPicker(
        mseed_list=mseed_files,
        waveform_dir=f'{data_dir}/mseed/{year}',
        phasenet_picks_df=picks_df,
        motion_model=diting_model,
        correct_pn_picks = -0.05, # correct the PhaseNet picks by this amount
    )
    par = input("Run in parallel? (y/n): ").lower()
    if par == "y":
        pol_pick_df = dmp.assign_polarity_parallel()
        # pol_pick_df.to_csv(f'{out_dir}/{year}/{year}_phasenet_picks.csv', index=False)
    elif par == "n":
        pol_pick_df = dmp.assign_polarity()
        # pol_pick_df.to_csv(f'{out_dir}/{year}/{year}_phasenet_picks.csv', index=False)
    else:
        print("Invalid input. Exiting...")
        break
    
    print(f'{len(pol_pick_df) - pol_pick_df["diting_polarity"].isna().sum()} valid polarity assigned')
    pol_pick_df.sort_values(by=['event_time', 'file_name']).to_csv(f'{out_dir}/{year}/{year}_phasenet_picks_pol.csv', index=False)

    break
pol_pick_df

Processing year: 2008
Running parallel processing with 8 cores


Polarity Detection:   0%|          | 0/151 [00:00<?, ?it/s]

2643 valid polarity assigned


,station_id,begin_time,phase_index,phase_time,phase_score,phase_type,file_name,phase_amplitude,phase_amp,event_id,event_time,pn_quality,diting_polarity,diting_sharpness
9,XQ.ME39.01.BH,2008-01-28T20:06:13.900,3866,2008-01-28 20:06:52.560,0.922,P,nc40210243.mseed,2182.706787,2182.706787,nc40210243,2008-01-28 20:06:43.900,1,1,I
0,NC.KCR..SH,2008-01-28T20:06:13.900,3571,2008-01-28 20:06:49.610,0.942,P,nc40210243.mseed,332.947723,332.947723,nc40210243,2008-01-28 20:06:43.900,1,1,E
6,XQ.ME08.01.BH,2008-01-28T20:06:13.900,3963,2008-01-28 20:06:53.530,0.943,P,nc40210243.mseed,3155.247559,3155.247559,nc40210243,2008-01-28 20:06:43.900,1,-1,I
7,XQ.ME09.01.BH,2008-01-28T20:06:13.900,4539,2008-01-28 20:06:59.290,0.950,P,nc40210243.mseed,775.828247,775.828247,nc40210243,2008-01-28 20:06:43.900,1,1,I
4,NC.LGP..SH,2008-01-28T20:06:13.900,4308,2008-01-28 20:06:56.980,0.965,P,nc40210243.mseed,646.003662,646.003662,nc40210243,2008-01-28 20:06:43.900,1,-1,E
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2635,NC.KBO..HH,2008-12-30T13:45:50.775,5027,2008-12-30 13:46:41.045,0.931,P,nc71180461.mseed,5293.184082,5293.184082,nc71180461,2008-12-30 13:46:20.770,1,-1,E
2636,NC.KOM..SH,2008-12-30T13:45:50.775,5038,2008-12-30 13:46:41.155,0.939,P,nc71180461.mseed,381.807739,381.807739,nc71180461,2008-12-30 13:46:20.770,1,1,E
2637,NC.KRMB..HH,2008-12-30T13:45:50.775,4563,2008-12-30 13:46:36.405,0.966,P,nc71180461.mseed,2860.719482,2860.719482,nc71180461,2008-12-30 13:46:20.770,1,1,I
2641,XQ.ME03.01.BH,2008-12-30T13:45:50.775,4703,2008-12-30 13:46:37.805,0.977,P,nc71180461.mseed,1977.267700,1977.267700,nc71180461,2008-12-30 13:46:20.770,1,1,I


# Combine with NCEDC picks (if not in NCEDC picks) and Convert to PyRocko format `txt`

In [8]:
%reload_ext autoreload
%autoreload 2
from classes_functions.NCEDC_files_converter import NCEDC_HypoInverse_reader
    
# 
for year in years_dirs:
    print(f'Processing year: {year}')
    nc_picks_df = pd.read_csv(f'{nc_picks_dir}/{year}_NCEDC_picks_above_slab.csv')
    nc_picks_df['color'] = 0
    pn_picks_pol = pd.read_csv(f'{out_dir}/{year}/{year}_phasenet_picks_pol.csv')
    pn_picks_pol['station_id'] = pn_picks_pol['station_id'].astype(str) + 'Z'

    before = nc_picks_df.shape[0]
    # print(nc_picks_df.columns)
    # print(pn_picks_pol.columns)

    new_picks = []
    for event_id, event_nc_df in nc_picks_df.groupby('event_id'):
        event_pn_df = pn_picks_pol[pn_picks_pol.event_id == event_id]
        # picks not available in NCEDC get them from PhaseNet-DiTing
        # print(event_nc_df.station_id, event_nc_df.shape, event_pn_df.shape)
        # print(event_pn_df.station_id)
        for i, row in event_pn_df.iterrows():
            if row.station_id not in event_nc_df.station_id.values:
                # NCEDC picks are in U/D format, PhaseNet-DiTing picks are in 1/-1 format
                polarity = row.diting_polarity
                new_picks.append({
                    'event_id': row.event_id,
                    'etime': event_nc_df.etime.values[0],
                    'elat': event_nc_df.elat.values[0],
                    'elon': event_nc_df.elon.values[0],
                    'edep': event_nc_df.edep.values[0],
                    'emag': event_nc_df.emag.values[0],
                    'station_id': row.station_id,
                    'phase_type': row.phase_type,
                    'phase_polarity': polarity,
                    'phase_time': row.phase_time,
                    'color': 1, # red to identify PhaseNet-DiTing picks
                })

    nc_picks_df = pd.concat(
        [nc_picks_df, pd.DataFrame(new_picks)], ignore_index=True
        ).sort_values(by=['phase_time', 'event_id']).reset_index(drop=True)            
    nc_picks_df.to_csv(f'{year}_NCEDC_picks_above_slab_PN.csv', index=False)
    
    after = nc_picks_df.shape[0]
    print(f'Before: {before}, After: {after}, Added: {after-before} picks for {year}...')
    
    ##### Convert to Pyrocko format
    nchypoinv = NCEDC_HypoInverse_reader()
    nc_pick_txt = nchypoinv.df_to_pyrocko_marker(
        picks_df=nc_picks_df,
        output_dir=f'{nc_picks_dir}',
        file_name=f'{year}_NCEDC_picks_above_slab_pyrocko_PN.txt',
        color_row=True,
    )
    print(f'Done for {year}...')
    break
print(nc_pick_txt[:10000])
nc_picks_df.phase_polarity.value_counts()

Processing year: 2008
Before: 2577, After: 3949, Added: 1372 picks for 2008...
Done for 2008...
# Snuffler Markers File Version 0.2
event: 2008-01-28 20:06:43.9000  0 NbrnTP3fAbnFbmOHnKYaXRvj7uf=   40.5875000 -123.6515000         22.66 1.78 None  nc40210243 None
phase: 2008-01-28 20:06:49.0500  0 NC.KCS..SHZ      NbrnTP3fAbnFbmOHnKYaXRvj7uf= 2008-01-28 20:06:43.9000 P             1 False
phase: 2008-01-28 20:06:49.5900  0 NC.KCR..SHZ      NbrnTP3fAbnFbmOHnKYaXRvj7uf= 2008-01-28 20:06:43.9000 P             1 False
phase: 2008-01-28 20:06:50.1000  1 XQ.ME30.01.BHZ   NbrnTP3fAbnFbmOHnKYaXRvj7uf= 2008-01-28 20:06:43.9000 P            -1 False
phase: 2008-01-28 20:06:50.8400  0 NC.KHMB..HHZ     NbrnTP3fAbnFbmOHnKYaXRvj7uf= 2008-01-28 20:06:43.9000 P            -1 False
phase: 2008-01-28 20:06:51.5500  0 NC.KHBB..HHZ     NbrnTP3fAbnFbmOHnKYaXRvj7uf= 2008-01-28 20:06:43.9000 P             1 False
phase: 2008-01-28 20:06:51.7500  0 NC.KPP..SHZ      NbrnTP3fAbnFbmOHnKYaXRvj7uf= 2008-01-28 20:06

phase_polarity
 1.0    2157
-1.0    1576
Name: count, dtype: int64